<a href="https://www.nvidia.com/dli"> <img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;"> </a>

# The Surge of Agents with the NeMo Agent Toolkit: Integrating Multiple AI Frameworks

## Introduction

Welcome to this hands-on tutorial on integrating multiple AI frameworks with the NeMo Agent Toolkit! In this notebook, we'll transform the "Surge of Agents" example into an NeMo Agent Toolkit workflow that orchestrates four powerful frameworks:

- **OpenAI Python Library**: For direct access to language models with a clean API
- **LangChain**: For structured data handling and composable processing chains
- **LangGraph**: For graph-based workflow management with modular components
- **CrewAI**: For collaborative multi-agent orchestration with specialized roles

The key insight of this tutorial is that **you don't need to rewrite your existing code** to benefit from the NeMo Agent Toolkit's orchestration capabilities. Instead, we'll create thin wrapper components that integrate your framework-specific code into a unified workflow.

By the end of this notebook, you'll understand how to:
1. Create NeMo Agent Toolkit components that wrap existing framework code
2. Configure a workflow that connects these components
3. Run the workflow as a unified system

Let's get started!

## Setup and Configuration

Before diving into the frameworks, we need to establish our development environment. We'll configure access to the NVIDIA AI Foundation Models platform, which provides access to powerful open-source models like Llama 3.1.

### Key Configuration Elements:

- **API Key**: The authentication token required to access NVIDIA's API services. In production environments, this should be stored securely as an environment variable rather than hardcoded in your notebooks.

- **Model Selection**: We're using `meta/llama-3.1-70b-instruct`, a powerful open-source LLM that balances performance and efficiency contained in an NVIDIA NIM.

Let's begin by setting up these configuration parameters:

In [ ]:
import os

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

model_name = "meta/llama-3.1-70b-instruct"

### Phase 1: Project Setup

Now we'll create the project structure for our NeMo Agent Toolkit workflow. We'll use the NeMo Agent Toolkit CLI to create a new workflow and set up the necessary directories.

In [ ]:
# Create the workflows directory if it doesn't exist
!mkdir -p workflows

# Create a new NeMo Agent Toolkit workflow
!nat workflow create --no-install --workflow-dir workflows surge_of_agents

# Create additional directories for configs and data
!mkdir -p workflows/surge_of_agents/configs
!mkdir -p workflows/surge_of_agents/data

Let's examine the project structure that was created:

In [ ]:
# List the project structure
!tree workflows/surge_of_agents

The NeMo Agent Toolkit CLI has created a basic project structure with:
- `pyproject.toml`: Package configuration file
- `src/surge_of_agents/`: Source directory for our components
  - `__init__.py`: Package initialization file
  - `register.py`: Component registration file
  - `surge_of_agents_function.py`: Default NeMo Agent Toolkit component file

Now, let's update the `pyproject.toml` file to include the dependencies we need for our multi-framework integration:

In [ ]:
%%writefile workflows/surge_of_agents/pyproject.toml
[build-system]
build-backend = "setuptools.build_meta"
requires = ["setuptools >= 64"]

[project]
name = "surge_of_agents"
version = "0.1.0"
dependencies = [
  "nvidia-nat[langchain,llama-index,crewai]",
  "langchain_nvidia_ai_endpoints",
  "pydantic",
]
requires-python = ">=3.12"
description = "NeMo Agent Toolkit workflow integrating multiple AI frameworks"
classifiers = ["Programming Language :: Python"]

[project.entry-points."nat.components"]
surge_of_agents = "surge_of_agents.register"

## Phase 2: Framework Component Wrappers

Now we'll create wrapper components for each of the four frameworks. These wrappers will allow us to integrate existing framework-specific code into our NeMo Agent Toolkit workflow without rewriting it.

### 1. OpenAI Wrapper Component

First, let's create a wrapper for the OpenAI Python library that generates math equations. This component will:
1. Use the OpenAI client to access the NVIDIA API
2. Generate a math equation suitable for pre-algebra students
3. Return the equation as output

Let's create this component:

In [ ]:
%%writefile workflows/surge_of_agents/src/surge_of_agents/openai_wrapper.py
import logging
import os
from typing import Dict, Any

from nat.builder.builder import Builder
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)

class EquationGeneratorConfig(FunctionBaseConfig, name="equation_generator"):
    """Configuration for the equation generator component."""
    model_name: str = "meta/llama-3.1-70b-instruct"
    temperature: float = 0.5

@register_function(config_type=EquationGeneratorConfig)
async def equation_generator(config: EquationGeneratorConfig, builder: Builder):
    """
    A wrapper for the OpenAI Python library that generates math equations.
    """
    from openai import OpenAI
    import os

    async def _generate_equation(student_level: str) -> Dict[str, Any]:
        """
        Generate a math equation using the OpenAI API.
        
        Args:
            student_level: The student level of the equation, such as "pre-algebra", "algebra", "geometry", or "calculus"
            
        Returns:
            A dictionary containing the generated equation
        """
        # Initialize the OpenAI client
        client = OpenAI(
            organization="nvidia",
            base_url="https://integrate.api.nvidia.com/v1",
            api_key=os.getenv("NVIDIA_API_KEY"),
        )
        
        # Create the prompt based on student_level
        prompt = f"""
        Create a math equation suitable for a {student_level} student that involves solving for a single variable, x.
        Provide only the equation, like "3x - 5 = 10".
        """
        
        # Generate the equation
        response = client.chat.completions.create(
            model=config.model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=config.temperature
        )
        
        # Extract and return the equation
        equation = response.choices[0].message.content.strip()
        return {"equation": equation}

    yield FunctionInfo.from_fn(
        _generate_equation,
        description="Generates math equations of varying difficulty levels using the OpenAI API"
    )

### 2. LangChain Wrapper Component

Next, let's create a wrapper for LangChain that generates word problems from equations. This component will:
1. Use LangChain's prompt templates and output parsers
2. Create a structured workflow using the pipeline operator
3. Return a word problem that matches the given equation

Let's create this component:

In [ ]:
%%writefile workflows/surge_of_agents/src/surge_of_agents/langchain_wrapper.py
import logging
from typing import Dict, Any

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)

class WordProblemGeneratorConfig(FunctionBaseConfig, name="word_problem_generator"):
    """Configuration for the word problem generator component."""
    llm_name: LLMRef
    debug_mode: bool = False

@register_function(config_type=WordProblemGeneratorConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def word_problem_generator(config: WordProblemGeneratorConfig, builder: Builder):
    """
    A wrapper for LangChain that generates word problems from equations.
    """
    from langchain.globals import set_debug
    from langchain.output_parsers import PydanticOutputParser
    from langchain_core.prompts import PromptTemplate
    from pydantic import BaseModel, Field
    
    # Enable debug mode if requested
    if config.debug_mode:
        set_debug(True)
    
    # Get the LLM from the builder
    llm = await builder.get_llm(config.llm_name, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
    
    # Define a structured data model for word problems
    class WordProblem(BaseModel):
        word_problem: str = Field(description="The text of the math word problem")
    
    # Create a parser that will extract structured data from LLM responses
    word_problem_parser = PydanticOutputParser(pydantic_object=WordProblem)
    
    # Define a template for generating word problems with instructions for proper formatting
    word_problem_prompt = PromptTemplate.from_template(
        """Given the equation {equation}, create a realistic word problem that matches it.
        The problem should involve a real-world scenario (e.g., shopping, travel) and require solving for x.
        Provide only the word problem.
        Format your response as JSON: {format_instructions}. Do not include any other text but the JSON.""",
        partial_variables={"format_instructions": word_problem_parser.get_format_instructions()}
    )
    
    # Compose the entire workflow as a chain using the pipeline operator
    chain = word_problem_prompt | llm | word_problem_parser
    
    async def _generate_word_problem(equation: str) -> Dict[str, Any]:
        """
        Generate a word problem from an equation using LangChain.
        
        Args:
            equation: The math equation to convert into a word problem
            
        Returns:
            A dictionary containing the equation and generated word problem
        """
        # Execute the chain with the equation
        result = await chain.ainvoke({"equation": equation})
        
        # Return the equation and word problem
        return {
            "equation": equation,
            "word_problem": result.word_problem
        }
    
    yield FunctionInfo.from_fn(
        _generate_word_problem,
        description="Generates word problems from math equations using LangChain"
    )

### 3. LangGraph Wrapper Component

Now, let's create a wrapper for LangGraph that solves the equation and explains the solution. This component will:
1. Define nodes for solving and explaining
2. Create a graph with connections between nodes
3. Execute the graph to process the equation and word problem

Let's create this component:

In [ ]:
%%writefile workflows/surge_of_agents/src/surge_of_agents/langgraph_wrapper.py
import logging
from typing import Dict, Any

from nat.builder.builder import Builder
from nat.builder.framework_enum import LLMFrameworkEnum
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.component_ref import LLMRef
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)

class EquationSolverConfig(FunctionBaseConfig, name="equation_solver"):
    """Configuration for the equation solver component."""
    llm_name: LLMRef
    debug_mode: bool = False

@register_function(config_type=EquationSolverConfig, framework_wrappers=[LLMFrameworkEnum.LANGCHAIN])
async def equation_solver(config: EquationSolverConfig, builder: Builder):
    """
    A wrapper for LangGraph that solves equations and explains the solutions.
    """
    from langchain.globals import set_debug
    from langchain_core.prompts import PromptTemplate
    from langgraph.graph import Graph, START
    
    # Enable debug mode if requested
    if config.debug_mode:
        set_debug(True)
    
    # Get the LLM from the builder
    llm = await builder.get_llm(config.llm_name, wrapper_type=LLMFrameworkEnum.LANGCHAIN)
    
    # Create prompts for solving and explaining
    equation_solver_prompt = PromptTemplate(
        input_variables=["equation", "word_problem"],
        template="""Given the equation {equation} and matching word problem {word_problem}, solve it by providing only the mathematical steps as a list.
        Each part should be a single equation or expression, showing the progression to the final solution, without any explanatory text. For example, for "5 + x = 13", output:
        5 + x = 13 -> x = 13 - 5 -> x = 8"""
    )
    
    equation_solution_explainer_prompt = PromptTemplate(
        input_variables=["equation", "word_problem", "solution"],
        template="""Given the equation {equation}, the matching word problem {word_problem}, and the solution {solution}, explain the solution in plain English using the fewest words possible."""
    )
    
    # Create chains
    equation_solver_chain = equation_solver_prompt | llm
    equation_solution_explainer_chain = equation_solution_explainer_prompt | llm
    
    # Define node functions
    def equation_solver_node(input_dict):
        equation = input_dict["equation"]
        word_problem = input_dict["word_problem"]
        solution = equation_solver_chain.invoke({"equation": equation, "word_problem": word_problem})
        # Ensure solution is a string (extract content if it's an AIMessage)
        if hasattr(solution, 'content'):
            solution = solution.content
        return {"solution": solution, "equation": equation, "word_problem": word_problem}
    
    def equation_solution_explainer_node(input_dict):
        equation = input_dict["equation"]
        word_problem = input_dict["word_problem"]
        solution = input_dict["solution"]
        explanation = equation_solution_explainer_chain.invoke({"equation": equation, "word_problem": word_problem, "solution": solution})
        # Ensure explanation is a string (extract content if it's an AIMessage)
        if hasattr(explanation, 'content'):
            explanation = explanation.content
        return {"explanation": explanation, "equation": equation, "word_problem": word_problem, "solution": solution}
    
    async def _solve_and_explain(equation: str, word_problem: str) -> Dict[str, Any]:
        """
        Solve an equation and explain the solution using LangGraph.
        
        Args:
            equation: The math equation to solve
            word_problem: The word problem that matches the equation
            
        Returns:
            A dictionary containing the equation, word problem, solution, and explanation
        """
        # Create our workflow graph
        graph = Graph()
        
        # Add our processing nodes
        graph.add_node("Solve Equation", equation_solver_node)
        graph.add_node("Explain Solution", equation_solution_explainer_node)
        
        # Define the flow between nodes
        graph.add_edge(START, "Solve Equation")
        graph.add_edge("Solve Equation", "Explain Solution")
        
        # Set the finish point to the last node so its output is returned
        graph.set_finish_point("Explain Solution")
        
        # Compile the graph into a runnable workflow
        workflow = graph.compile()
        
        # Run the workflow with our input data
        workflow_result = workflow.invoke({
            "equation": equation,
            "word_problem": word_problem
        })

        print(workflow_result)
        
        # Return the workflow result
        return workflow_result
    
    yield FunctionInfo.from_fn(
        _solve_and_explain,
        description="Solves math equations and provides step-by-step explanations using LangGraph"
    )

### 4. CrewAI Wrapper Component

Finally, let's create a wrapper for CrewAI that reviews the solution. This component will:
1. Create specialized agents with distinct roles and goals
2. Define tasks for each agent to perform
3. Coordinate their collaboration through a sequential workflow

Let's create this component:

In [ ]:
%%writefile workflows/surge_of_agents/src/surge_of_agents/crewai_wrapper.py
import logging
import os
from typing import Dict, Any

from nat.builder.builder import Builder
from nat.builder.function_info import FunctionInfo
from nat.cli.register_workflow import register_function
from nat.data_models.function import FunctionBaseConfig

logger = logging.getLogger(__name__)

class SolutionReviewerConfig(FunctionBaseConfig, name="solution_reviewer"):
    """Configuration for the solution reviewer component."""
    model_name: str = "meta/llama-3.1-70b-instruct"
    verbose: bool = True

@register_function(config_type=SolutionReviewerConfig)
async def solution_reviewer(config: SolutionReviewerConfig, builder: Builder):
    """
    A wrapper for CrewAI that reviews math solutions.
    """
    from crewai import Agent, Task, Crew, LLM
    import os
    
    async def _review_solution(equation: str, word_problem: str, solution: str, explanation: str) -> Dict[str, Any]:
        """
        Review a math solution using CrewAI.
        
        Args:
            equation: The math equation
            word_problem: The word problem that matches the equation
            solution: The step-by-step solution
            explanation: The explanation of the solution
            
        Returns:
            A dictionary containing the review results
        """
        # Initialize the LLM with the correct format for CrewAI
        llm = LLM(
            model=f"nvidia_nim/{config.model_name}", 
            api_key=os.getenv("NVIDIA_API_KEY")
        )
        
        # Agent 1: Accuracy Checker
        accuracy_checker_agent = Agent(
            role="Accuracy Checker",
            goal="Verify the mathematical correctness of the word problem, equation, and solution steps",
            backstory="You are a meticulous mathematician with a keen eye for detail. Your expertise lies in ensuring that every calculation and logical step in a math problem is correct, leaving no room for errors. You double-check solutions against the original problem to confirm accuracy.",
            llm=llm,
            verbose=config.verbose
        )
        
        # Define the accuracy checking task
        accuracy_task = Task(
            description=f"Review the following: word problem '{word_problem}', equation '{equation}', and solution '{solution}'. Verify that the solution steps correctly solve the equation and match the word problem. Output 'Correct' if accurate, or identify any errors if incorrect.",
            expected_output="A concise statement confirming accuracy ('Correct') or detailing any errors found.",
            agent=accuracy_checker_agent,
        )
        
        # Agent 2: Clarity Reviewer
        clarity_reviewer_agent = Agent(
            role="Clarity Reviewer",
            goal="Ensure the word problem and solution explanation are clear, engaging, and educationally valuable for students",
            backstory="You are an experienced educator with a passion for making math accessible and engaging. You excel at evaluating whether problems and explanations are easy to understand, appropriately challenging, and relevant to students' learning needs.",
            llm=llm,
            verbose=config.verbose,
        )
        
        # Define the clarity review task
        clarity_task = Task(
            description=f"Review the following: word problem '{word_problem}' and solution explanation '{explanation}'. Assess if they are clear, engaging, and suitable for middle school students. Provide feedback, including at least one suggestion for improvement if applicable.",
            expected_output="A brief assessment of clarity and educational value, plus one suggestion for enhancement.",
            agent=clarity_reviewer_agent,
        )
        
        # Create a crew with both agents and their tasks
        crew = Crew(
            agents=[accuracy_checker_agent, clarity_reviewer_agent], 
            tasks=[accuracy_task, clarity_task], 
            process="sequential",  # Tasks will be executed in order 
            verbose=config.verbose
        )
        
        # Execute the full workflow
        result = crew.kickoff()
        
        # Return the review results
        return {
            "equation": equation,
            "word_problem": word_problem,
            "solution": solution,
            "explanation": explanation,
            "review": result
        }
    
    yield FunctionInfo.from_fn(
        _review_solution,
        description="Reviews math solutions for accuracy and clarity using CrewAI"
    )

### 5. Updating the Register File

Now that we've created all our wrapper components, we need to update the `register.py` file to import them:

In [ ]:
%%writefile workflows/surge_of_agents/src/surge_of_agents/register.py
# pylint: disable=unused-import
# flake8: noqa

# Import all wrapper components
from surge_of_agents.openai_wrapper import equation_generator
from surge_of_agents.langchain_wrapper import word_problem_generator
from surge_of_agents.langgraph_wrapper import equation_solver
from surge_of_agents.crewai_wrapper import solution_reviewer

__all__ = [
    "equation_generator",
    "word_problem_generator",
    "equation_solver",
    "solution_reviewer"
]

In this section, we'll implement a structured sequential workflow that explicitly manages the data flow between components. This approach ensures proper data passing between tools and provides a clear execution sequence.

## Phase 3: Running Our Multiframework Workflow

### 1. Installing

Now let's install our package and test the sequential workflow:

In [ ]:
%uv pip install -e workflows/surge_of_agents

### 2. Creating the Workflow Configuration

Now we'll create a configuration file that defines our sequential workflow. First, create a directory for the config:

In [ ]:
!mkdir -p workflows/surge_of_agents/configs

Now we'll write the config file using the various frameworks we wrapped as NeMo Agent Toolkit components above:

In [ ]:
%%writefile workflows/surge_of_agents/configs/sequential_config.yml
general:
  uvloop: true
  telemetry:
    tracing:
      phoenix:
          _type: phoenix
          endpoint: http://localhost:6006/v1/traces
          project: surge_of_agents

llms:
  nim_llm:
    _type: nim
    model_name: meta/llama-3.1-70b-instruct
    temperature: 0.7
    max_tokens: 1000
    
functions:
  equation_generator:
    _type: equation_generator
    llm_name: nim_llm
    
  word_problem_generator:
    _type: word_problem_generator
    llm_name: nim_llm
    
  equation_solver:
    _type: equation_solver
    llm_name: nim_llm

  solution_reviewer:
    _type: solution_reviewer
    llm_name: nim_llm

workflow:
  _type: react_agent
  llm_name: nim_llm
  system_prompt: |
    You are a helpful assistant that follows a sequential workflow to create and solve math problems.

    You need to perform the following steps in order:
    1. Generate a math equation
    2. Create a word problem based on the equation
    3. Solve the equation
    4. Review the solution

    You have access to the following tools:

    {tools}

    You may respond in one of two formats.
    Use the following format exactly to ask the human to use a tool:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action (if there is no required input, include "Action Input: None")  
    Observation: wait for the human to respond with the result from the tool, do not assume the response

    ... (this Thought/Action/Action Input/Observation can repeat N times. If you do not need to use a tool, or after asking the human to use any tools and waiting for the human to respond, you might know the final answer.)
    Use the following format once you have the final answer:

    Thought: I now know the final answer
    Final Answer: the equation, word problem, solution, and explanation
  
  tool_names:
    - equation_generator
    - word_problem_generator
    - equation_solver
    - solution_reviewer
  verbose: true
  retry_parsing_errors: true
  max_retries: 10

### 3. Starting Phoenix for Observability

Now we can start Phoenix and open the Phoenix UI to see the profiling data. We have instrumented our config for it.

In [ ]:
import subprocess
import time

# Start the Phoenix server using Popen to gain direct control over the process
# We also suppress the output by redirecting stdout and stderr
phoenix_process = subprocess.Popen(
    ["phoenix", "serve"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

# Give Phoenix a moment to start up
time.sleep(3)

print(f"Phoenix server started with PID: {phoenix_process.pid}")
print("You can access the Phoenix UI at: http://localhost:6006")

### 4. Running the Workflow

Now, at long last, we can run our workflow.

In [ ]:
!nat run --config_file workflows/surge_of_agents/configs/sequential_config.yml --input "Create a math problem about for pre-algebra students"

### 5. Stopping the Phoenix Server

Let's clean up by stopping the Phoenix server we started. In a production system, you would not stop Phoenix.


In [ ]:
# Check if the process object exists and is running
if 'phoenix_process' in locals() and phoenix_process.poll() is None:
    print(f"Stopping Phoenix server with PID: {phoenix_process.pid}...")
    
    # Send the termination signal to the process
    phoenix_process.terminate()
    
    try:
        # Wait for the process to terminate
        phoenix_process.wait(timeout=5)
        print("Phoenix server stopped successfully.")
    except subprocess.TimeoutExpired:
        # If it doesn't terminate gracefully, force kill it
        print("Server did not terminate gracefully. Forcing kill...")
        phoenix_process.kill()
        phoenix_process.wait()
        print("Phoenix server killed.")
    
    # Clean up the variable
    del phoenix_process
else:
    print("Phoenix server was not running or the process object was not found.")


## Conclusion

In this notebook, we've demonstrated how to integrate multiple AI frameworks using the NeMo Agent Toolkit.

This approach showcases the benefits of the NeMo Agent Toolkit:

- **Framework Interoperability**: Seamlessly integrate OpenAI, LangChain, LangGraph, and CrewAI.
- **Workflow Management**: Orchestrate complex workflows with proper data handling.
- **Deployment Options**: Deploy workflows as APIs, CLIs, or web applications.
- **Extensibility**: Easily add new components or modify existing ones.